In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import csv
import os
import sys
import time

In [2]:
pd.set_option("display.max_rows",400)

In [3]:
driver = webdriver.Chrome('./chromedriver')

### 幣種選擇
* 人民幣(CNY)
* 美元(USD)
* 澳幣(AUD)
* 南非幣(ZAR)
* 紐西蘭幣(NZD)
* 歐元(EUR)
* 港幣(HKD)
* 日圓(JPY)
* 墨西哥披索(MXN)
* 加幣(CAD)
* 瑞士法郎(CHF)
* 英鎊(GBP)
* 瑞典幣(SEK)
* 新加坡幣(SGD)
* 泰銖(THB)

In [4]:
print("Insert currency:\nSelect list as follow\n人民幣(CNY)\n美元(USD)\n澳幣(AUD)\n南非幣(ZAR)\n紐西蘭幣(NZD)\n歐元(EUR) \
\n港幣(HKD)\n日圓(JPY)\n墨西哥披索(MXN)\n加幣(CAD)\n瑞士法郎(CHF)\n英鎊(GBP) \
\n瑞典幣(SEK)\n新加坡幣(SGD)\n泰銖(THB)")
currency = str.upper(input())

Insert currency:
Select list as follow
人民幣(CNY)
美元(USD)
澳幣(AUD)
南非幣(ZAR)
紐西蘭幣(NZD)
歐元(EUR) 
港幣(HKD)
日圓(JPY)
墨西哥披索(MXN)
加幣(CAD)
瑞士法郎(CHF)
英鎊(GBP) 
瑞典幣(SEK)
新加坡幣(SGD)
泰銖(THB)
GBP


### 即期或現鈔

* 即期輸入:spot
* 現鈔輸入:cash


In [5]:
print("Insert spot or cash:")
rate_type = str.lower(input())

Insert spot or cash:
spot


### 時間

* 當日輸入:Now
* 3個月輸入:3Months
* 6個月輸入:6Months
* 1年輸入:1Year


In [6]:
print("Insert duration:\nSelect list as follow\n當日輸入:Now\
\n3個月輸入:3Months\
\n6個月輸入:6Months\
\n1年輸入:1Year")
duration = str.upper(input())

Insert duration:
Select list as follow
當日輸入:Now
3個月輸入:3Months
6個月輸入:6Months
1年輸入:1Year
3months


In [7]:
if rate_type == "spot":
     type_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/div/img[2]"
elif rate_type == "cash":
     type_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/div/img[1]"

In [8]:
if duration == "NOW":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[1]"
elif duration == "3MONTHS":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[2]"
elif duration == "6MONTHS":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[3]"
elif duration == "1YEAR":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[4]"

In [9]:
# Data page
driver.get("https://www.esunbank.com.tw/bank/personal/deposit/rate/forex/exchange-rate-chart")

time.sleep(1)

In [10]:
# Select the currency 
select_currency_arrow = driver.find_element_by_xpath("/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/ul/li/span")
select_currency_arrow.click()

# Select the currency
select_currency = driver.find_elements_by_css_selector(".transformSelectDropdown li span")
for i in select_currency:
    if currency in i.text:
        i.click()
        break

time.sleep(1)

In [11]:
# Select Spot or Cash
currency_type = driver.find_element_by_xpath(type_html)
currency_type.click()

time.sleep(1)

In [12]:
# Select the duration
currency_duration = driver.find_element_by_xpath(duration_html)
currency_duration.click()

time.sleep(1)

In [13]:
# Select data type
data_type = driver.find_element_by_xpath("/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[3]/div/label[2]")
data_type.click()

time.sleep(1)

In [14]:
# Click search button
search_button = driver.find_element_by_xpath('/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[4]/a')
search_button.click()

time.sleep(1)

In [15]:
df = pd.DataFrame(columns=["Date", "Bid_Rate", "Offer_Rate"])
pages = True

while pages:
    items = driver.find_elements_by_css_selector("#inteTable tbody tr")

    for item in items:
        tds = item.find_elements_by_css_selector("td")

        if tds[0].get_attribute("class") != "itemTtitle":
            continue

        df = df.append({
                    "Date": tds[0].text,
                    "Bid_Rate":  tds[1].text,
                    "Offer_Rate":  tds[2].text
                    }, ignore_index=True)

    nextBtn = driver.find_element_by_css_selector(".pageNumberBlock .down")
    if "active" in nextBtn.get_attribute("class"):
        nextBtn.click()
    else:
        pages = False

time.sleep(4)

In [16]:
# Close the browser
driver.quit()

In [17]:
df["Offer_Rate"] = df["Offer_Rate"].astype(float)
df["Bid_Rate"] = df["Bid_Rate"].astype(float)

In [25]:
df_min = df[df.Offer_Rate == df.Offer_Rate.min()]
df_max = df[df.Offer_Rate == df.Offer_Rate.max()]

In [26]:
print(df)
print("")
print('Bid rate average in {} : {}'.format(str.lower(duration),round(df["Bid_Rate"].mean(),2)))
print('Offer rate average in {} : {}'.format(str.lower(duration),round(df["Offer_Rate"].mean(),2)))
print("")
print("The lowest price :")
print(df_min.to_string(index=False))
print("")
print("The highest price :")
print(df_max.to_string(index=False))
print("")

          Date  Bid_Rate  Offer_Rate
0   2020-01-20     38.72       39.12
1   2020-01-21     38.99       39.39
2   2020-01-22     39.20       39.60
3   2020-01-30     39.40       39.80
4   2020-01-31     39.51       39.91
5   2020-02-03     39.25       39.65
6   2020-02-04     39.05       39.45
7   2020-02-05     38.84       39.24
8   2020-02-06     38.71       39.11
9   2020-02-07     38.80       39.20
10  2020-02-10     38.67       39.07
11  2020-02-11     38.75       39.15
12  2020-02-12     38.69       39.09
13  2020-02-13     38.91       39.31
14  2020-02-14     38.89       39.29
15  2020-02-15     38.94       39.34
16  2020-02-17     38.84       39.24
17  2020-02-18     39.03       39.43
18  2020-02-19     38.83       39.23
19  2020-02-20     38.74       39.14
20  2020-02-21     39.16       39.56
21  2020-02-24     39.14       39.54
22  2020-02-25     39.20       39.60
23  2020-02-26     39.06       39.46
24  2020-02-27     38.79       39.19
25  2020-03-02     38.26       38.66
2